# Spark Streaming - Socket TCP

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import col
from pyspark.sql.functions import length
from pyspark.sql.functions import lower

1)El primer paso es crear una sessión Spark.

In [2]:
spark = SparkSession.builder.appName("PEC2StructuredWordCount").getOrCreate()

2)A continuación, creamos un un Streaming DataFrame que represente los datos de texto recibidos desde un servidor, escuchando en el puerto localhost:9998
Para ello abrimos una terminal y ejecutamos el comando -> **nc -lk 9998**
El DataFrame **lines** representa una tabla no vinculada conteniendo los datos de texto en streaming. La tabla contiene una columna de cadenas, llamada "value", y cada línea en los datos de texto del streaming se convierte en una fila en la tabla. 

In [3]:
lines =spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()

3)El siguiente paso, es convertir el DataFrame en un Dataset, que contenga todas las palabras. Para ello, utilizamos la función split utilizando como separador el espacio. Con la función explode toma el array que contiene las palabras y los separa en múltiples líneas. 

In [4]:
words =lines.select(explode(split(lines.value, " ")).alias("word"))

4)Una vez obtenidas las palabras, pasamos a realizar un filtrado de estas siguiendo las especificaciones de la práctica.
Como primer paso, antes de realizar el filtrado de las palabras, convertimos todos los caracteres de cada palabra a minúscula. El primer filtrado siguiendo las especificaciones será por lo tanto, filtrar las palabras que presentan una longitud menor a 4 caracteres. El segundo filtrado es para las palabras *como, cuando, donde, pero, hasta, mira, mientras* 
El último filtrado es para las palabras que empiezan por *co*

In [5]:
filteredWords = words.withColumn('word', lower(col("word"))) \
                     .filter(length(col("word")) > 3) \
                     .filter(~col("word").isin(["como","cuando","donde","pero","hasta","mira","mientras"]) ) \
                     .filter(~col("word").startswith('co'))

5)Una vez filtradas las palabras, definimos un DataFrame agrupando por valores únicos en el DataSet y los contamos. Este Streaming DataFrame representa el word counts en ejecución del stream.

In [6]:
wordCounts =filteredWords.groupBy("word").count()

6)Como último paso, preparamos la consulta de los datos del streaming. Utilizamos la opción *outputMode("complete")* para indicarle que imprima en la consola el conjunto completo de recuentos cada vez que se produce una actualización. Y comenzamos la computación del streaming con **start()**
Para evitar que el proceso salga mientras la consulta o query esté activa, usamos *awaitTermination()*

In [ ]:
# Start running the query that prints the running counts to the console
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()